# loop on the OCs to estimate isochrone and mass

In [1]:
using PyCall

## PopStar modules...
synthetic= pyimport("popstar.synthetic")
evolution= pyimport("popstar.evolution")
atmospheres= pyimport("popstar.atmospheres")
reddening= pyimport("popstar.reddening")

using Distributions, Statistics, Distances
using DataFrames
using Random , Printf, Glob

import PyPlot , CSV
import Distances, StatsBase

rootdir = ENV["GAIA_ROOT"]

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/plotdens"
ocdir= "$wdir/oc2"
isodir= "$wdir/isochrones"

cd(wdir)

In [2]:
### functions

function mass_isochrone(f)
    
    fname= f[1:end-4] ; fvot= f[1:end-7] 
    ocfile= "$ocdir/$fname.csv" ; voname= "$fvot.vot"
    ocmassfile= "$ocdir/$fname-mass.csv"
    
    df= CSV.read(ocfile , delim= ";")
    
    t= split(fvot,"-")
    if length(t) == 3
        t[1]= t[1]*"-"*t[2]
    end
    ocname= t[1]
    
    println("## Name: $fname")
    logage= df1[(df1.name .== ocname), :log_age]
    # println(logage[1])
    
    ################################
    ## ISOCHRONES ##################
    ################################
    agmedian= median(df.ag[.! isnan.(df.ag)])
    distance= median(df.distance)
    nstar= length(df.distance)
    
    println("## N(star): $nstar")
    # Define isochrone parameters
    logAge= logage[1]  # Age in log(years)
    AKs= 0.0 # extinction in mags
    dist= distance # distance in parsec
    metallicity= 0.0 # Metallicity in [M/H]
    
    println("### log Age: $logAge")
    println("### AKs: $AKs")
    println("### distance: $dist")
    println("### metallicity: $metallicity")
    
    ## Define evolution/atmosphere models and extinction law
    if logAge>6.5
        evo_model = evolution.MergedBaraffePisaEkstromParsec() 
    else
        evo_model= evolution.MISTv1()
    end
    
    atm_func = atmospheres.get_merged_atmosphere
    red_law = reddening.RedLawHosek18b()

    ## Filter
    filt_list = [ "gaia,dr2_rev,Gbp", "gaia,dr2_rev,G"  ,"gaia,dr2_rev,Grp"]
    
    #####################################
    println("### Computing isochrones..")
    iso = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list, iso_dir= isodir)
    
    println("### Updating the DF and creating the new one with stellar mass..")
    
    data= iso.points
    println(data.keys())
    ndata= size(data)[1]

    mass= zeros(ndata)
    Gmag= zeros(ndata)
    BPmag= zeros(ndata)
    RPmag= zeros(ndata)
    color= zeros(ndata)

    for i in 1:ndata
        mass[i]= data[i][4]
        Gmag[i]= data[i][10]
        BPmag[i]= data[i][9]
        RPmag[i]= data[i][11]
        color[i]= BPmag[i]-RPmag[i]
    end

    ## Cluster
    BmR= df.bp .- df.rp
    GMAG= df.gbar + 5 .* log10.(df.distance) .- 17.
    

    ### Stellar mass 
    ################
    nxy= length(mass)
    A= Array{Float64}(undef,2,nxy)
    for i in 1:nxy
        A[1,i]= color[i]
        A[2,i]= Gmag[i]
    end
    
    d= Distances.Euclidean()
    mstar= zeros(length(BmR))

    for i in 1:length(BmR)
        starm= [BmR[i]; GMAG[i]]
        r= Distances.colwise(d, A, starm)
    
        idx= argmin(r)
        mstar[i]= mass[idx]
    end
    
    df.mass= mstar
    
    ## Some Logs mass stats...
    massmean= mean(mstar)
    println("### Mass (mean): $massmean")
    
    ### save the OC df
    CSV.write(ocmassfile, df, delim=";") 
    println("\n\n")    
end

mass_isochrone (generic function with 1 method)

In [ ]:
cd(ocdir)
files= glob("*-oc.csv")
cd(wdir)

sample= rootdir*"/master/notebooks/data/BrowseTargets.18292.1530479692.fake"
fileres= "isolist-done.csv"

df1= CSV.read(sample, delim= "|")
df1.name= strip.(df1.name)

noc= 0
for f in files
    filered= CSV.read(fileres,delim=";",copycols= true)
    
    if !(f in filered.fileoc)
        println(f)
        mass_isochrone(f)
        push!(filered,(f,))
        CSV.write(fileres,filered,delim=";")
    end
    
end

Blanco 1-11.8deg-oc.csv
## Name: Blanco 1-11.8deg-oc
## N(star): 409
### log Age: 7.75
### AKs: 0.0
### distance: 238.14678177016
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]


┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:72
└ @ Main ./In[2]:72
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:72
└ @ Main ./In[2]:72
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:73
└ @ Main ./In[2]:73
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:73
└ @ Main ./In[2]:73
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:74
└ @ Main ./In[2]:74
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = mass_isochrone(::String) at In[2]:74
└ @ Main ./In[2]:74
┌ Warning: `getindex(o::PyObject, 

### Mass (mean): 0.48822860635696813



Bochum 9-1.0deg-oc.csv
## Name: Bochum 9-1.0deg-oc
## N(star): 111
### log Age: 8.09
### AKs: 0.0
### distance: 452.1277618501667
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.9384872432432434



C1331-622-1.0deg-oc.csv
## Name: C1331-622-1.0deg-oc
## N(star): 80
### log Age: 9.25
### AKs: 0.0
### distance: 264.4380300813956
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.33321085000000006



CBJC 4-1.0deg-oc.csv
## Name: CBJC 4-1.0deg-oc
## N(star): 269
### log Age: 7.7
### AKs: 0.0
### distance: 886.0385719281551
### metallicity: 0.0
### Computing isochrones..
### Updatin

M =   1.250 Msun  T =  6491 K  m_hst_Grp = 11.30### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.7422463235294118



Cep OB4-3.8deg-oc.csv
## Name: Cep OB4-3.8deg-oc
## N(star): 374
### log Age: 6.1
### AKs: 0.0
### distance: 996.1628000692169
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.931439449458891



Chamaleon I-1.8deg-oc.csv
## Name: Chamaleon I-1.8deg-oc
## N(star): 86
### log Age: 6.0
### AKs: 0.0
### distance: 190.27003645962628
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mas

Changing to logg=2.00 for T=  9266 logg=1.50### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.9354396261682242



Collinder 135-2.6deg-oc.csv
## Name: Collinder 135-2.6deg-oc
## N(star): 543
### log Age: 7.6
### AKs: 0.0
### distance: 294.6927159844501
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.5621548802946592



Collinder 140-2.5deg-oc.csv
## Name: Collinder 140-2.5deg-oc
## N(star): 152
### log Age: 7.7
### AKs: 0.0
### distance: 388.7165355508283
### metallicity: 0.0
### Computing isochrones..

Changing to logg=2.00 for T=  9283 logg=1.50
Changing to logg=2.00 for T=  9303 logg=1.50
Changing to logg=2.00 for T=  9326 logg=1.51
Changing to logg=2.00 for T=

...

Changing to logg=5.00 for T= 57253 logg=6.12### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.8438570671378093



NGC 2281-2.0deg-oc.csv
## Name: NGC 2281-2.0deg-oc
## N(star): 322
### log Age: 8.785
### AKs: 0.0
### distance: 529.3992658914555
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.9133386987577641



NGC 2287-2.2deg-oc.csv
## Name: NGC 2287-2.2deg-oc
## N(star): 482
### log Age: 8.25
### AKs: 0.0
### distance: 730.8919389128897
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (m

     Starting at:  2020-01-02 17:58:27.326618### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.7040460638297872



NGC 2306-1.1deg-oc.csv
## Name: NGC 2306-1.1deg-oc
## N(star): 111
### log Age: 9.0
### AKs: 0.0
### distance: 283.8680154727879
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.6374463693693694



NGC 2318-1.0deg-oc.csv
## Name: NGC 2318-1.0deg-oc
## N(star): 122
### log Age: 8.75
### AKs: 0.0
### distance: 1396.7230398766794
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (m

Starting filter: gaia,dr2_rev,Grp   Elapsed time: 121.13 seconds### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.4342358108108106



NGC 2358-1.5deg-oc.csv
## Name: NGC 2358-1.5deg-oc
## N(star): 115
### log Age: 8.785
### AKs: 0.0
### distance: 1098.0939368429006
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.2521957217391302



NGC 2360-1.5deg-oc.csv
## Name: NGC 2360-1.5deg-oc
## N(star): 337
### log Age: 8.65
### AKs: 0.0
### distance: 1103.706266543
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst

Excessive output truncated after 541923 bytes.





NGC 2362-1.3deg-oc.csv
## Name: NGC 2362-1.3deg-oc
## N(star): 216
### log Age: 6.64
### AKs: 0.0
### distance: 1312.138406287541
### metallicity: 0.0
### Computing isochrones..